In [1]:
# !conda install -c conda-forge ipyleaflet --yes
# !jupyter nbextension enable --py --sys-prefix ipyleaflet
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter labextension install jupyter-leaflet
# from ipyleaflet import *

import folium
from folium import plugins

import requests
import random
import matplotlib.pyplot as plt
import math
import time
import json
import datetime
import dateutil.parser
from tqdm.notebook import tqdm

def send_request(url, data, method):
    headers = {"content-type": "application/json"}
    if method == 'PUT': res = requests.put(url, data=json.dumps(data), headers=headers)
    if method == 'POST': res = requests.post(url, data=json.dumps(data), headers=headers)
    return res

def getUserTaskDistSq(user, task):
#     print(user, task)
    dist = (user[0] - task[0]) ** 2 + (user[1] - task[1]) ** 2
    return dist

def timeLimitExceeded(a):
    assignedTime = dateutil.parser.isoparse(a['assignedTime'])
    dt = datetime.datetime.now(datetime.timezone.utc)
    now = dt.replace(tzinfo=datetime.timezone.utc)
    if assignedTime + datetime.timedelta(minutes=a['timeLimit']) < now: return True
    return False

def users2matrix(users):
    """
    returns user matrices, where matrix = [lat, long, unknown, idle, waiting, busy, avg walking speed, completion ratio, username]
    """
    matrix = []
    for user in users:
        arr = [0] * 9
        arr[0] = user['location']['lat']
        arr[1] = user['location']['lng']
        status = user['status']
        if status == 'unknown': arr[2] = 1
        if status == 'idle': arr[3] = 1
        if status == 'waiting': arr[4] = 1
        if status == 'busy': arr[5] = 1
        arr[6] = user['avgWalkingSpeed']
        assignments = user['assignment']
        assignments.reverse() # latest aasignments first
        assignments = assignments[:min(100, len(assignments))] # gets the first 100
        ratio = 0
        for assignment in assignments:
            if assignment['isCompleted']: ratio += 1
        if (len(assignments)): ratio /= len(assignments)
        arr[7] = ratio
        arr[8] = user['username']
        matrix.append(arr)
    return matrix


def tasks2matrix(tasks):
    """
    returns task matrices, matrix = [lat, long, targetAoI, num_user, _id]
    """
    matrix = []
    for task in tasks:
        arr = [0] * 5
        arr[0] = task['trashbin']['location']['lat']
        arr[1] = task['trashbin']['location']['lng']
        arr[2] = task['targetAoI']
        num_users = 0
        for a in task['assignment']:
                if not a['isValid']: continue
                if a['isCompleted']: continue
                if timeLimitExceeded(a): continue
                num_users += 1
        arr[3] = num_users
        arr[4] = task['_id']
        matrix.append(arr)
    return matrix


In [31]:

BASE_URL = "http://localhost:3000/api/v0" # dev
# BASE_URL = "https://crowd-sensing.herokuapp.com/api/v0" # test
CLOCK = 5
ASSIGN_EVERY_N_CYCLES = 3
COLORS = ['blue', 'red', 'purple', 'darkpurple', 'darkblue', 'green']

class Service:
    def __init__(self, num_timeslots):
        super().__init__()
        self.num_timeslots = num_timeslots
        self.userLocations = {}
        
    def getAllTasks(self):
        url = f'{BASE_URL}/tasks/all'
        res = requests.get(url)
        users = res.json()['data']['tasks']
        return users

    def getAllUsers(self):
        url = f'{BASE_URL}/users/all'
        res = requests.get(url)
        users = res.json()['data']['users']
        return users
    
    def assignNearestTask(self, user, tasks):
        nearestTask = None
        minDist = 1e9
        for task in tasks:
            dist = getUserTaskDistSq(user, task)
            if dist < minDist:
                nearestTask = task
                minDist = dist
        
        url = f'{BASE_URL}/tasks/assign'
        data = {
            "username": user[-1],
            "taskID": nearestTask[-1],
        }
        send_request(url, data, 'POST')

    def assignTasks(self, users, tasks):
        for user in users:
                self.assignNearestTask(user, tasks)
                
    def plotUsers(self):
        mp = folium.Map(location=[49.26252990000112, -123.25006520184665], zoom_start=14)
        for task in self.tasks:
            folium.Marker(
                location=[task['trashbin']['location']['lat'], task['trashbin']['location']['lng']], # coordinates for the marker (Earth Lab at CU Boulder)
#                 popup=user['username'], # pop-up label for the marker
                size=1,
                width=1,
                icon=folium.Icon(color='lightgray', icon='cube', prefix='fa')
            ).add_to(mp)
        for k, v in self.userLocations.items():
            folium.PolyLine(v, color=random.choice(COLORS), weight=2.5, opacity=1).add_to(mp)
        display(mp)
    
    def recordUserMoves(self):
        for user in self.users:
            _id = user['_id']
            location = (user['location']['lat'], user['location']['lng'])
            if _id in self.userLocations:
                self.userLocations[_id].append(location)
            else:
                self.userLocations[_id] = [location]
    
    def run(self):
        for cycle in tqdm(range(self.num_timeslots)):
            self.users = self.getAllUsers()
            self.tasks = self.getAllTasks()
            
            userMatrix = users2matrix(self.users)
            self.recordUserMoves()
            
#             if cycle % ASSIGN_EVERY_N_CYCLES == 0:
# taskMatrix = tasks2matrix(self.tasks)
#             self.tasks = self.getAllTasks()
#             self.assignTasks(userMatrix, taskMatrix)
            time.sleep(CLOCK)
        self.plotUsers()
            

In [32]:
service = Service(num_timeslots=10)
service.run()


In [83]:
ubcMap = Map(
                center=[49.26252990000112, -123.25006520184665],
                zoom=14
            )
ubcMap

Map(center=[49.26252990000112, -123.25006520184665], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [17]:
for _ in range(5):
    print(random.choice(COLORS))

darkred
orange
darkblue
gray
darkred
